In [2]:
# imports
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
%load_ext tensorboard 

C:\Users\USER\anaconda3\envs\chesnn\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\USER\anaconda3\envs\chesnn\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [4]:
#!taskkill /IM "tensorboard.exe" /F
!rmdir /S /Q %temp%\.tensorboard-info
%tensorboard --logdir=runs --host localhost

In [3]:
# data params
data_params = {'batch_size':128, 
               'shuffle':True}

In [15]:
import os 

# dataset 
class Dataset(torch.utils.data.Dataset):
  def __init__(self, samples, list_IDs, labels):
        'Initialization'
        self.samples = samples
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

         # Load data and get label
        X = self.samples[ID]
        y = self.labels[ID]
        return X, y
    
samples = torch.load('../data/X.pt')
partition = {'train': list(range(17000)), 'dev': list(range(17000, 18000)), 'test': []}
labels = torch.load('../data/Y.pt')
# data loader
training_set = Dataset(samples, partition['train'], labels)
training_generator = torch.utils.data.DataLoader(training_set, **data_params)

validation_set = Dataset(samples, partition['dev'], labels)
validation_generator = torch.utils.data.DataLoader(validation_set, **data_params)

In [16]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
# nn model
class EloNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(50, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 25),
            torch.nn.ReLU(),
            torch.nn.Linear(25, 2),
        )
    
    def forward(self, ml_in):
        return self.layers(ml_in)
    
model = EloNN()
model.to(device)

EloNN(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=25, bias=True)
    (3): ReLU()
    (4): Linear(in_features=25, out_features=2, bias=True)
  )
)

In [45]:
# hyper params
learning_rate=1e-3
epochs=2000 # 2000
l2=1e-5

In [20]:
# helper function
def get_num_correct(predicted, actual):
    count = 0
    for i, p in enumerate(predicted):
        if abs(p[0] - actual[i][0]) < 100 and abs(p[1] - actual[i][1]) < 100:
            count += 1
    return count

In [ ]:
# train model
tb = SummaryWriter()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2)
criterion = torch.nn.MSELoss(reduction='mean')

tb = SummaryWriter()
has_graphed = False

for epoch in range(epochs):

    total_loss = 0
    total_correct = 0

    for inputs, labels in training_generator:
        inputs = inputs.float()
        labels = labels.float()
        inputs, labels = inputs.to(device), labels.to(device)
        preds = model(inputs)
        if epoch == 0 and not has_graphed:
            tb.add_graph(model, inputs)
            has_graphed = True
        
        loss = criterion(preds, labels)
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct / len(training_generator), epoch)

    tb.add_histogram("linear1.bias", model.layers[0].bias, epoch)
    tb.add_histogram("linear1.weight", model.layers[0].weight, epoch)
    tb.add_histogram("linear2.bias", model.layers[2].bias, epoch)
    tb.add_histogram("linear2.weight", model.layers[2].weight, epoch)
    tb.add_histogram("linear3.bias", model.layers[4].bias, epoch)
    tb.add_histogram("linear3.weight", model.layers[4].weight, epoch)
        
    tb.add_hparams(
        {"lr": learning_rate, "bsize": data_params['batch_size'], "shuffle": data_params['shuffle']},
        {
            "accuracy": total_correct / len(training_generator),
            "loss": total_loss,
        },
    )

    print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)
    
tb.close()
torch.save(model.state_dict(), 'multi_layer_perceptron.pt')

epoch: 0 total_correct: 55 loss: 444388811.25
epoch: 1 total_correct: 308 loss: 206671034.9375
epoch: 2 total_correct: 407 loss: 154432508.8125
epoch: 3 total_correct: 446 loss: 132563249.375
epoch: 4 total_correct: 425 loss: 117660954.4375
epoch: 5 total_correct: 506 loss: 105157968.3125
epoch: 6 total_correct: 549 loss: 94223101.71875
epoch: 7 total_correct: 563 loss: 84900925.9375
epoch: 8 total_correct: 613 loss: 77404941.25
epoch: 9 total_correct: 692 loss: 72130842.375
epoch: 10 total_correct: 712 loss: 68587395.0625
epoch: 11 total_correct: 766 loss: 65563893.46875
epoch: 12 total_correct: 787 loss: 63159546.09375
epoch: 13 total_correct: 833 loss: 60866292.125
epoch: 14 total_correct: 863 loss: 58185053.8125
epoch: 15 total_correct: 948 loss: 56478744.609375
epoch: 16 total_correct: 984 loss: 54376457.328125
epoch: 17 total_correct: 1032 loss: 52118418.5625
epoch: 18 total_correct: 1004 loss: 50360074.953125
epoch: 19 total_correct: 1027 loss: 48221942.96875
epoch: 20 total_cor